In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Firefox()

In [ ]:
#####Buscar as urls das notícias e salvar no arquivo urls_lupa.tsv:

import csv
import math

#Lista da quantidade de notícias de cada mês para cada ano. Usado para saber quantas páginas tem
qtdPorAno = {
    2021: [68, 63, 72, 65, 97, 41],
    2020: [56, 50, 89, 169, 175, 181, 130, 109, 88, 168, 244, 70]
}

#Dado que uma página com a lista de notícias está aberta, retorna essa lista, com [url, data, vinheta, título, chamada] de cada notícia.
def getLinks():
    blocos = driver.find_elements_by_xpath('//div[@class="internaPGN"]/div')
    links = []
    for bloco in blocos:
        bloco_meta = bloco.find_element_by_class_name('bloco-meta').text
        meta = bloco_meta.split('|')
        date = meta[0].strip()
        vinheta = meta[2].strip()    
        bloco_chamada = bloco.find_element_by_xpath('.//h3[@class="bloco-chamada"]/a')
        url = bloco_chamada.get_attribute("href")
        title = bloco_chamada.get_attribute("title")
        chamada = bloco_chamada.text
        links.append({'url': url, 'date': date, 'vinheta': vinheta, 'title': title, 'chamada' : chamada})
    return links


#Entra em cada página e salva a lista das notícias no arquivo
with open('urls_lupa.tsv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['url', 'date', 'vinheta', 'title', 'chamada']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter='\t')
    writer.writeheader()    

    for ano in qtdPorAno.keys():
        qtdPorMes = qtdPorAno[ano]
        for i, qtd in enumerate(qtdPorMes):
            mes = str(i+1).zfill(2)    
            print(ano, mes, qtd)
            paginas = math.ceil(qtd/10)
            for pagina in range(paginas):
                page_group = 'https://piaui.folha.uol.com.br/lupa/'+str(ano)+'/'+mes+'/'+'page/'+str(pagina + 1)+'/'
                driver.get(page_group)
                links = getLinks()
                for link in links:
                    writer.writerow(link)


In [3]:
from selenium.common.exceptions import NoSuchElementException

from pathlib import Path
import csv

def saveLupaPage(filePath, claim):
    """! Acessa a página da notícia e salva a notícia completa
    @param filePath - Caminho do arquivo que será salvo
    @param claim - um dicionário com as informações principais da notícia (precisa ter os atributos url, date, vinheta, title, chamada).
      Isso foi lido do arquivo urls_lupa.tsv
    """
    url = claim['url']
    driver.get(url)

    with open(filePath, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['row', 'url', 'date', 'vinheta', 'title', 'chamada', 'contentType','tags', 'content']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter='\t')
        writer.writeheader()
        try:
            claim['contentType'] = driver.find_element_by_class_name('post-categ-label').text
        except NoSuchElementException:
            claim['contentType'] = 'NOTFOUND'
        claim['content'] = driver.find_element_by_class_name('post-inner').text
        elem_tags = driver.find_elements_by_xpath('//div[@class="post-tags no-print"]/ul[2]/li')
        tags = []
        for tag in elem_tags:
            tags.append(tag.text)
        claim['tags'] = ';'.join(tags)
        writer.writerow(claim)
    return claim

In [ ]:
# Lê a lista de urls de notícias, verifica se ela já foi salva, e se não foi, salva em um arquivo.
# Cada notícia será salva num arquivo diferente, para facilitar a verificação de qual já foi salva.
claims = [] 
path = 'lupa_files/'
Path(path).mkdir(exist_ok=True) #parents=True, 
with open('urls_lupa.tsv', 'r', encoding='utf-8') as f:
    read_urls = csv.DictReader(f, delimiter="\t", skipinitialspace=True)
    for i, row in enumerate(read_urls):
        print(i)
        print(row)
        row['row'] = i
        url = row['url']
        #Gera o nome do arquivo em fução da url
        filePath = Path(path + url.replace("https://piaui.folha.uol.com.br/lupa/", "").replace("/", "_")+".tsv")
        if not filePath.is_file():            
            claims.append(saveLupaPage(filePath, row))

In [ ]:
#Gerar lista de arquivos ruins: spanish ou não checagem

path = 'lupa_files/'
for file in listdir(path):
    full_path = join(path, file)
    print(full_path)
    with open(full_path, 'r', encoding='utf-8') as f:
        read_claims = csv.DictReader(f, delimiter="\t", skipinitialspace=True)
        for rclaims in read_claims:
            print(rclaims['url'])
            if rclaims['url'] not in bad_urls and isFactChecking(rclaims['content']):
                claims.append(rclaims)

In [ ]:
# Após salvar todas as páginas, essa parte verifica quais das páginas salvas realmente possuem alguma checagem de fatos, e junta elas num arquivo só (allLupaFiles.tsv)

from os import listdir
from os.path import join
import csv


def isFactChecking(text):
    # Nem todas as páginas possuem checagem de fatos. Só possuem as que tiverem as seguintes classes
    classes = ['VERDADEIRO, MAS', 'VERDADEIRO', 'EXAGERADO', 'FALSO', 'AINDA É CEDO PARA DIZER', 'CONTRADITÓRIO', 'SUBESTIMADO', 'INSUSTENTÁVEL', 'DE OLHO']
    for classe in classes:
        if classe in text:
            return True
    return False

def isSpanish(claim):
    spanish_words = ['Es falso', 'Es falsa', ' lo ', ' la ']
    for word in spanish_words:
        if word in claim['title'] or word in claim['chamada']:
            return True
    return False

# A lista de páginas que serão consideradas
claims = []

# Algumas páginas estão em espanhol, ou não precisam ser salvas. Coloquei as urls dessas notícias no arquivo lupa_ruins.txt para ignorá-las.
bad_urls = []
with open('bases/lupa_ruins.txt', 'r') as f:
    for i in f:
        bad_urls.append(i.strip())

list_spanish = []
list_not_factchecking = []

path = 'bases/lupa_files/'
for file in listdir(path):
    full_path = join(path, file)
    #print(full_path)
    with open(full_path, 'r', encoding='utf-8') as f:
        read_claims = csv.DictReader(f, delimiter="\t", skipinitialspace=True)
        for rclaims in read_claims:
            #print(rclaims['url'])
            if not isFactChecking(rclaims['content']):
                list_not_factchecking.append(rclaims['url'])
            elif isSpanish(rclaims):
                list_spanish.append(rclaims)
            else:
                if rclaims['url'] not in bad_urls:
                    claims.append(rclaims)

for i in list_spanish:
    print('spanish:', i['url'], i['title'])

print(len(claims))

with open('bases/allLupaFiles.tsv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['row', 'url', 'date', 'vinheta', 'title', 'chamada', 'contentType','tags', 'content']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter='\t')
    writer.writeheader()
    for i, claim in enumerate(claims):
        claim['row'] = i
        writer.writerow(claim)